In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -U langchain-community
!pip install --upgrade langchain
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [4]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SequentialChain


In [5]:
openai_llm_1 = OpenAI(
    model_name="ft:gpt-3.5-turbo-1106:pes:finetune-feedback:AQwJEq7T:ckpt-step-800",  # our fine tuned model
    openai_api_key="open_api_key"
)


/tmp/ipykernel_30/3633502107.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  openai_llm_1 = OpenAI(


In [6]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY="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"
LANGCHAIN_API_KEY="langchain_api_key"
LANGCHAIN_PROJECT="feedback_system"


In [7]:
# Sample essay written by an 8th grader
essay_text_1 = """Working in a hospital emergency room is very challenging for doctors and nurses. One of the main problems they face is handling many patients with different kinds of injuries and illnesses all at once. Sometimes, patients come in with very serious injuries and need immediate care, so doctors have to make quick decisions. Another issue is the risk of infection. Nurses and doctors have to be very careful to wash their hands, wear gloves, and use masks to avoid spreading germs. They also have to work very long hours, which can be exhausting. Because of these challenges, working in an emergency room requires skill, bravery, and a strong commitment to helping people."""

essay_prompt_1= "Based on the information you know, describe the main challenges faced by doctors and nurses working in a hospital emergency room and explain why these challenges are important."

# Scores
scores_1 = {
  'score': 0.85,               # Overall score for quality
  'content': 0.8,              # Quality and accuracy of information
  'prompt_adherence': 0.9,     # How well the essay addresses the prompt
  'language': 0.7,             # Quality of language used
  'narrativity': 0.6           # Engagement and storytelling
}


In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# Define your essay and prompt text
essay_text = "During the construction of the Empire State Building the builders experience many obstacles when trying to allow dirigibles to dock there..."
essay_prompt = "Based on the excerpt, describe the obstacles builders of the Empire State Building faced when attempting to allow dirigibles to dock."
flattened_scores = {
    "overall_score": 0.75,
    "content_score": 0.5,
    "organization_score": -1,
    "word_choice_score": -1,
    "sentence_fluency_score": -1,
    "conventions_score": -1,
    "prompt_adherence_score": 0.5,
    "language_score": 0.5,
    "narrativity_score": 0.5
}

# Initialize OpenAI LLM (replace 'openai_llm_1' with your OpenAI API key setup)
# openai_llm_1 = OpenAI(api_key="your_openai_api_key")

# Define the CoT prompt template
cot_prompt_template = PromptTemplate(
    input_variables=[
        "essay", "prompt", 
        "overall_score", "content_score", "organization_score", 
        "word_choice_score", "sentence_fluency_score", "conventions_score", 
        "prompt_adherence_score", "language_score", "narrativity_score"
    ],
    template="""
Essay Prompt: {prompt}
Essay: {essay}

You are a helpful assistant generating constructive feedback for essays based on specified scores (ranging from 0 to 1, where 1 is excellent and 0 is the least satisfactory). Follow the chain-of-thought reasoning to structure your response step-by-step, considering each aspect logically before providing feedback.

Begin with each parameter in the following order and address it based on its score (omit parameters with a score of -1):

1. **Overall Essay Score** ({overall_score}): Start with an overview, describing the general quality of the essay, considering the overall coherence, engagement, and relevance.
2. **Content** ({content_score}): Reflect on the essay’s depth and relevance, identifying strengths and offering ways to enhance the content if needed.
3. **Organization** ({organization_score}): Evaluate the structure, discussing whether ideas flow logically and where improvement in organization may enhance clarity.
4. **Word Choice** ({word_choice_score}): Assess vocabulary usage for precision and effectiveness, suggesting more fitting or expressive alternatives if beneficial.
5. **Sentence Fluency** ({sentence_fluency_score}): Analyze sentence variation and flow, recommending smoother transitions or variations where appropriate.
6. **Conventions** ({conventions_score}): Address grammar, punctuation, and spelling, noting any recurring errors and suggesting consistency improvements.
7. **Prompt Adherence** ({prompt_adherence_score}): Examine how closely the essay follows the prompt, suggesting ways to improve alignment if needed.
8. **Language** ({language_score}): Evaluate expressiveness and tone, noting how language choices enhance or detract from clarity and engagement.
9. **Narrativity** ({narrativity_score}): Consider the narrative flow, identifying ways to heighten reader engagement with sensory details or emotional resonance.

**Chain-of-Thought Reasoning Instructions**:
- For each parameter, reflect briefly on the strengths, followed by specific suggestions for enhancement.
- Provide coherent and cohesive feedback, integrating positive observations before constructive criticism.
- Avoid listing scores; instead, create a single flowing paragraph that incorporates each relevant point naturally.
- Aim for 150–180 words in total.

Generate feedback below in a coherent and constructive manner:
{{
    "Feedback": "The essay presents a well-considered account with some commendable strengths and areas for improvement..."
}}
    """
)

# Set up the feedback chain using the LLMChain
feedback_chain_2 = LLMChain(
    llm=openai_llm_1,
    prompt=cot_prompt_template
)

# Invoke the chain to get feedback
result = feedback_chain_2.invoke({
    "essay": essay_text,
    "prompt": essay_prompt,
    **flattened_scores
})

# Extract and print the feedback text
feedback_text = result.get('text', '')
print(feedback_text)


/tmp/ipykernel_30/2232751810.py:63: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  feedback_chain_2 = LLMChain(



The essay presents a well-considered account with some commendable strengths and areas for improvement. The content covers the obstacles faced during the Empire State Building's construction, yet it could be more comprehensive, especially regarding the specific challenges and potential solutions. The organization is clear but lacks depth, as the narrative progression could be more detailed and structured. Word choice is somewhat repetitive and could benefit from more precise terms, particularly when explaining the technical aspects of the construction. Sentence fluency is adequate, though the essay would benefit from more varied structures and smoother transitions between ideas. Conventions are mostly correct, but there are some minor issues with punctuation. The essay adheres closely to the prompt, but it could be more detailed in its description of the challenges. Language is appropriate for the topic, but it lacks the expressive depth that could make the essay more engaging. Overal